In [11]:

import scipy
#import pytrends
#from pytrends.request import TrendReq
from numpy import random
import matplotlib.pyplot as plt
from scipy import optimize
import sys
import time
import pandas as pd
import seaborn as sns
import numpy as np
from pandas.io.json import json_normalize
import json
from datetime import datetime, timedelta
from county_spending import get_county_spending
from ACS_data import colname_mask

pd.set_option('display.max_columns', None)

In [8]:
county_spending = get_county_spending()
covid_cases = pd.read_csv('covid_confirmed_usafacts.csv')
ACS_data = pd.read_csv('ACS_disability_by_county.csv', header=1)

/home/hank/Winter2021/Stat6367/ConsultingProject/county_spending.py:33: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  county_json = json_normalize(r.json())


In [15]:
# Whats up with all the missing counties in ACS_data. We need new ACS data anyway, so something to keep in mind
print(county_spending.shape)
print(covid_cases.shape)
print(ACS_data.shape)

(3233, 6)
(3193, 429)
(840, 209)


In [29]:
#county_spending = county_spending.rename(columns={'display_name':'county_name'})
county_spending['county_name'] = county_spending['county_name'].fillna('Missing')
#county_spending['county_name'] = county_spending['county_name'].apply(lambda x: x + ' County')
county_spending.head()


,amount,county_name,shape_code,population,per_capita,award_count
0,3.284831e+10,Los Angeles County,06037,10039107.0,3272.03,747836
1,8.909570e+09,Miami-Dade County,12086,2716940.0,3279.27,423598
2,2.713221e+10,Cook County,17031,5150233.0,5268.15,398169
3,1.348097e+10,Harris County,48201,4713325.0,2860.18,280146
4,6.607329e+09,Broward County,12011,1952778.0,3383.55,226534


In [30]:
covid_cases = covid_cases.rename(columns={'County Name':'county_name'})
covid_cases.head()

,countyFIPS,county_name,State,StateFIPS,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,2020-01-28,2020-01-29,2020-01-30,2020-01-31,2020-02-01,2020-02-02,2020-02-03,2020-02-04,2020-02-05,2020-02-06,2020-02-07,2020-02-08,2020-02-09,2020-02-10,2020-02-11,2020-02-12,2020-02-13,2020-02-14,2020-02-15,2020-02-16,2020-02-17,2020-02-18,2020-02-19,2020-02-20,2020-02-21,2020-02-22,2020-02-23,2020-02-24,2020-02-25,2020-02-26,2020-02-27,2020-02-28,2020-02-29,2020-03-01,2020-03-02,2020-03-03,2020-03-04,2020-03-05,2020-03-06,2020-03-07,2020-03-08,2020-03-09,2020-03-10,2020-03-11,2020-03-12,2020-03-13,2020-03-14,2020-03-15,2020-03-16,2020-03-17,2020-03-18,2020-03-19,2020-03-20,2020-03-21,2020-03-22,2020-03-23,2020-03-24,2020-03-25,2020-03-26,2020-03-27,2020-03-28,2020-03-29,2020-03-30,2020-03-31,2020-04-01,2020-04-02,2020-04-03,2020-04-04,2020-04-05,2020-04-06,2020-04-07,2020-04-08,2020-04-09,2020-04-10,2020-04-11,2020-04-12,2020-04-13,2020-04-14,2020-04-15,2020-04-16,2020-04-17,2020-04-18,2020-04-19,2020-04-20,2020-04-21,2020-04-22,2020-04-23,2020-04-24,2020-04-25,2020-04-26,2020-04-27,2020-04-28,2020-04-29,2020-04-30,2020-05-01,2020-05-02,2020-05-03,2020-05-04,2020-05-05,2020-05-06,2020-05-07,2020-05-08,2020-05-09,2020-05-10,2020-05-11,2020-05-12,2020-05-13,2020-05-14,2020-05-15,2020-05-16,2020-05-17,2020-05-18,2020-05-19,2020-05-20,2020-05-21,2020-05-22,2020-05-23,2020-05-24,2020-05-25,2020-05-26,2020-05-27,2020-05-28,2020-05-29,2020-05-30,2020-05-31,2020-06-01,2020-06-02,2020-06-03,2020-06-04,2020-06-05,2020-06-06,2020-06-07,2020-06-08,2020-06-09,2020-06-10,2020-06-11,2020-06-12,2020-06-13,2020-06-14,2020-06-15,2020-06-16,2020-06-17,2020-06-18,2020-06-19,2020-06-20,2020-06-21,2020-06-22,2020-06-23,2020-06-24,2020-06-25,2020-06-26,2020-06-27,2020-06-28,2020-06-29,2020-06-30,2020-07-01,2020-07-02,2020-07-03,2020-07-04,2020-07-05,2020-07-06,2020-07-07,2020-07-08,2020-07-09,2020-07-10,2020-07-11,2020-07-12,2020-07-13,2020-07-14,2020-07-15,2020-07-16,2020-07-17,2020-07-18,2020-07-19,2020-07-20,2020-07-21,2020-07-22,2020-07-23,2020-07-24,2020-07-25,2020-07-26,2020-07-27,2020-07-28,2020-07-29,2020-07-30,2020-07-31,2020-08-01,2020-08-02,2020-08-03,2020-08-04,2020-08-05,2020-08-06,2020-08-07,2020-08-08,2020-08-09,2020-08-10,2020-08-11,2020-08-12,2020-08-13,2020-08-14,2020-08-15,2020-08-16,2020-08-17,2020-08-18,2020-08-19,2020-08-20,2020-08-21,2020-08-22,2020-08-23,2020-08-24,2020-08-25,2020-08-26,2020-08-27,2020-08-28,2020-08-29,2020-08-30,2020-08-31,2020-09-01,2020-09-02,2020-09-03,2020-09-04,2020-09-05,2020-09-06,2020-09-07,2020-09-08,2020-09-09,2020-09-10,2020-09-11,2020-09-12,2020-09-13,2020-09-14,2020-09-15,2020-09-16,2020-09-17,2020-09-18,2020-09-19,2020-09-20,2020-09-21,2020-09-22,2020-09-23,2020-09-24,2020-09-25,2020-09-26,2020-09-27,2020-09-28,2020-09-29,2020-09-30,2020-10-01,2020-10-02,2020-10-03,2020-10-04,2020-10-05,2020-10-06,2020-10-07,2020-10-08,2020-10-09,2020-10-10,2020-10-11,2020-10-12,2020-10-13,2020-10-14,2020-10-15,2020-10-16,2020-10-17,2020-10-18,2020-10-19,2020-10-20,2020-10-21,2020-10-22,2020-10-23,2020-10-24,2020-10-25,2020-10-26,2020-10-27,2020-10-28,2020-10-29,2020-10-30,2020-10-31,2020-11-01,2020-11-02,2020-11-03,2020-11-04,2020-11-05,2020-11-06,2020-11-07,2020-11-08,2020-11-09,2020-11-10,2020-11-11,2020-11-12,2020-11-13,2020-11-14,2020-11-15,2020-11-16,2020-11-17,2020-11-18,2020-11-19,2020-11-20,2020-11-21,2020-11-22,2020-11-23,2020-11-24,2020-11-25,2020-11-26,2020-11-27,2020-11-28,2020-11-29,2020-11-30,2020-12-01,2020-12-02,2020-12-03,2020-12-04,2020-12-05,2020-12-06,2020-12-07,2020-12-08,2020-12-09,2020-12-10,2020-12-11,2020-12-12,2020-12-13,2020-12-14,2020-12-15,2020-12-16,2020-12-17,2020-12-18,2020-12-19,2020-12-20,2020-12-21,2020-12-22,2020-12-23,2020-12-24,2020-12-25,2020-12-26,2020-12-27,2020-12-28,2020-12-29,2020-12-30,2020-12-31,2021-01-01,2021-01-02,2021-01-03,2021-01-04,2021-01-05,2021-01-06,2021-01-07,2021-01-08,2021-01-09,2021-01-10,2021-01-11,2021-01-12,2021-01-13,2021-01-14,2021-01-15,

In [13]:
ACS_data = ACS_data.iloc[:,~ACS_data.columns.str.startswith('Margin')]
ACS_data = ACS_data.rename(columns = {'Geographic Area Name':'county_name'})
#rename_dict = colname_mask(ACS_data.columns[2:])
#ACS_data = ACS_data.rename(columns = rename_dict)
ACS_data.head()

,id,Geographic Area Name,tot,tot_M,tot_F,tot_W,tot_B,tot_I,tot_A,tot_P,tot_O,tot_T,tot_w,tot_H,tot_0-5,tot_5-17,tot_18-34,tot_35-64,tot_65-74,tot_75+,tot_aud,tot_aud_0-18,tot_aud_0-18_0-5,tot_aud_0-18_5-17,tot_aud_18-64,tot_aud_18-64_18-34,tot_aud_18-64_35-64,tot_aud_65+,tot_aud_65+_65-74,tot_aud_65+_75+,tot_vis,tot_vis_0-18,tot_vis_0-18_0-5,tot_vis_0-18_5-17,tot_vis_18-64,tot_vis_18-64_18-34,tot_vis_18-64_35-64,tot_vis_65+,tot_vis_65+_65-74,tot_vis_65+_75+,tot_cog,tot_cog_0-18,tot_cog_18-64,tot_cog_18-64_18-34,tot_cog_18-64_35-64,tot_cog_65+,tot_cog_65+_65-74,tot_cog_65+_75+,tot_amb,tot_amb_0-18,tot_amb_18-64,tot_amb_18-64_18-34,tot_amb_18-64_35-64,tot_amb_65+,tot_amb_65+_65-74,tot_amb_65+_75+,tot_self,tot_self_0-18,tot_self_18-64,tot_self_18-64_18-34,tot_self_18-64_35-64,tot_self_65+,tot_self_65+_65-74,tot_self_65+_75+,tot_ind,tot_ind_18-64,tot_ind_18-64_18-34,tot_ind_18-64_35-64,tot_ind_65+,tot_ind_65+_65-74,tot_ind_65+_75+,dis,dis_M,dis_F,dis_W,dis_B,dis_I,dis_A,dis_P,dis_O,dis_T,dis_w,dis_H,dis_0-5,dis_5-17,dis_18-34,dis_35-64,dis_65-74,dis_75+,dis_aud,dis_aud_0-18,dis_aud_0-18_0-5,dis_aud_0-18_5-17,dis_aud_18-64,dis_aud_18-64_18-34,dis_aud_18-64_35-64,dis_aud_65+,dis_aud_65+_65-74,dis_aud_65+_75+,dis_vis,dis_vis_0-18,dis_vis_0-18_0-5,dis_vis_0-18_5-17,dis_vis_18-64,dis_vis_18-64_18-34,dis_vis_18-64_35-64,dis_vis_65+,dis_vis_65+_65-74,dis_vis_65+_75+,dis_cog,dis_cog_0-18,dis_cog_18-64,dis_cog_18-64_18-34,dis_cog_18-64_35-64,dis_cog_65+,dis_cog_65+_65-74,dis_cog_65+_75+,dis_amb,dis_amb_0-18,dis_amb_18-64,dis_amb_18-64_18-34,dis_amb_18-64_35-64,dis_amb_65+,dis_amb_65+_65-74,dis_amb_65+_75+,dis_self,dis_self_0-18,dis_self_18-64,dis_self_18-64_18-34,dis_self_18-64_35-64,dis_self_65+,dis_self_65+_65-74,dis_self_65+_75+,dis_ind,dis_ind_18-64,dis_ind_18-64_18-34,dis_ind_18-64_35-64,dis_ind_65+,dis_ind_65+_65-74,dis_ind_65+_75+,%,%_M,%_F,%_W,%_B,%_I,%_A,%_P,%_O,%_T,%_w,%_H,%_0-5,%_5-17,%_18-34,%_35-64,%_65-74,%_75+,%_aud,%_aud_0-18,%_aud_0-18_0-5,%_aud_0-18_5-17,%_aud_18-64,%_aud_18-64_18-34,%_aud_18-64_35-64,%_aud_65+,%_aud_65+_65-74,%_aud_65+_75+,%_vis,%_vis_0-18,%_vis_0-18_0-5,%_vis_0-18_5-17,%_vis_18-64,%_vis_18-64_18-34,%_vis_18-64_35-64,%_vis_65+,%_vis_65+_65-74,%_vis_65+_75+,%_cog,%_cog_0-18,%_cog_18-64,%_cog_18-64_18-34,%_cog_18-64_35-64,%_cog_65+,%_cog_65+_65-74,%_cog_65+_75+,%_amb,%_amb_0-18,%_amb_18-64,%_amb_18-64_18-34,%_amb_18-64_35-64,%_amb_65+,%_amb_65+_65-74,%_amb_65+_75+,%_self,%_self_0-18,%_self_18-64,%_self_18-64_18-34,%_self_18-64_35-64,%_self_65+,%_self_65+_65-74,%_self_65+_75+,%_ind,%_ind_18-64,%_ind_18-64_18-34,%_ind_18-64_35-64,%_ind_65+,%_ind_65+_65-74,%_ind_65+_75+
0,0500000US01003,"Baldwin County, Alabama",220911,106961,113950,189325,17668,N,N,N,N,N,183610,N,10616,36210,40659,86588,28682,18156,(X),46826,10616,36210,127247,40659,86588,46838,28682,18156,(X),46826,10616,36210,127247,40659,86588,46838,28682,18156,(X),36210,127247,40659,86588,46838,28682,18156,(X),36210,127247,40659,86588,46838,28682,18156,(X),36210,127247,40659,86588,46838,28682,18156,(X),127247,40659,86588,46838,28682,18156,31901,15573,16328,27623,2050,N,N,N,N,N,27581,N,0,2538,3850,11393,6070,8050,10749,413,0,413,3431,384,3047,6905,2782,4123,5281,626,0,626,1623,650,973,3032,997,2035,10586,2052,5880,2745,3135,2654,739,1915,15885,0,7555,690,6865,8330,3533,4797,6181,553,2534,419,2115,3094,1030,2064,10780,4500,1200,3300,6280,2131,4149,14.4,14.6,14.3,14.6,11.6,N,N,N,N,N,15.0,N,0.0,7.0,9.5,13.2,21.2,44.3,4.9,0.9,0.0,1.1,2.7,0.9,3.5,14.7,9.7,22.7,2.4,1.3,0.0,1.7,1.3,1.6,1.1,6.5,3.5,11.2,5.0,5.7,4.6,6.8,3.6,5.7,2.6,10.5,7.6,0.0,5.9,1.7,7.9,17.8,12.3,26.4,2.9,1.5,2.0,1.0,2.4,6.6,3.6,11.4,6.2,3.5,3.0,3.8,13.4,7.4,22.9
1,0500000US01015,"Calhoun County, Alabama",111075,52442,58633,80403,24662,N,N,N,N,N,79067,N,6699,17635,24227,42369,12017,8128,(X),24334,6699,17635,66596,24227,42369,20145,12017,8128,(X),24334,6699,17635,66596,24227,42369,20145,12017,8128,(X),17635,66596,24227,42369,20145,12017,8128,(X),17635,66596,24227,42369,20145,12017,8128,(X),17635